In [1]:
import re
import json
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import nltk
from gensim.utils import simple_preprocess

In [2]:
years = ['2016'] # Hier die Jahre angeben, die geladen werden sollen (final: 2016-2023)
path = "/Users/jan/Documents/Promotion/BIBB GW/hdd/Pflege/"#Hier der Pfad zu den JSON-Dateien!
with open(f"{path}Pflege_2016.json", 'r') as file:
    output = json.load(file)
docs = [str(value) for value in output["conditions_description"].values()]
del output

In [3]:
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.
# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]
# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]
# Remove stop words
stop_words = set(stopwords.words('german'))
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]
docs = remove_stopwords(docs)

In [4]:
lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [5]:
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [6]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [7]:
# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [8]:
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -2.1362.
[([(0.025290394, 'stellenangebots'),
   (0.025268167, 'karte'),
   (0.021707818, 'tarifvertrag'),
   (0.018367987, 'teilzeit'),
   (0.017030776, 'stellen'),
   (0.015441479, 'vergütung'),
   (0.015395431, 'zusatzleistungen'),
   (0.015043824, 'findet'),
   (0.015001009, 'anwendung'),
   (0.014997802, 'besetzung'),
   (0.014916601, 'stellenangebotes'),
   (0.014912846, 'besetzung_stellenangebotes'),
   (0.01460242, 'arbeitsverhältnis'),
   (0.013823486, 'arbeitszeit'),
   (0.013500375, 'unbefristetes'),
   (0.013421387, 'unbefristetes_arbeitsverhältnis'),
   (0.013329232, 'tätigkeit'),
   (0.013285313, 'informationen'),
   (0.013202374, 'beginn'),
   (0.013174985, 'form')],
  -0.3281519880285761),
 ([(0.029090354, 'zwingend'),
   (0.022460496, 'vieles'),
   (0.020899257, 'zulagen'),
   (0.020492423, 'extraleistungen'),
   (0.019912325, 'wünschenswert'),
   (0.019489683, 'erhalten'),
   (0.01766463, 'plus'),
   (0.01746943, 'mehr'),
   (0.014868785, 'eur